##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用结构化剪枝的稀疏权重

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

模型中的结构化剪枝权重使模型在特定模式下稀疏，在合适的硬件支持下，这可以加速模型推断时间。

本教程将展示如何：

- 基于 MNIST 数据集定义并训练具有特定结构化稀疏性的模型
- 将剪枝后的模型转换为 TFLite 格式
- 呈现剪枝后权重的结构

有关用于模型优化的剪枝技术的一般概述，请参阅[剪枝概述](https://tensorflow.google.cn/model_optimization/guide/pruning)。有关一般权重剪枝的教程，请参阅 [Keras 中的剪枝](https://tensorflow.google.cn/model_optimization/guide/pruning/pruning_with_keras)。

## 权重的结构化剪枝

结构化剪枝在训练过程开始时系统地将模型权重归零。将这一剪枝技术应用于常规权重块可以加速支持硬件上的推断，例如：将模型中的权重按四个权重为一个块的方式分组，并将每个块中的两个权重归零，称为 *2 x 4* 缩减。此技术仅适用于由 TensorFlow Lite 转换的模型的权重张量的最后一个维度。例如，TensorFlow Lite 中的 `Conv2D` 层权重的结构为 `[channel_out, height, width, channel_in]`，`Dense` 层权重的结构为 `[channel_out, channel_in]`。稀疏性模式应用于最后一个维度的权重：`channel_in`。

与随机稀疏性相比，结构化稀疏性通常由于结构限制而具有较低的准确率，但是，它可以在支持的硬件上显著减少推断时间。

剪枝可以与其他模型压缩技术一起应用于模型，以获得更好的压缩率。请参阅[协作优化技术](https://blog.tensorflow.org/2021/10/Collaborative-Optimizations.html)中的量化和聚类示例，了解更多详细信息。

## 安装

准备开发环境和数据。

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization
! pip install -q matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## 从 [MNIST](https://tensorflow.google.cn/datasets/catalog/mnist) 数据集下载图像数据并归一化

In [ ]:
# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## 定义结构化剪枝参数

定义剪枝参数并指定结构化剪枝类型。将剪枝参数设置为 `(2, 4)`。这些设置意味着在一个包含四个元素的块中，至少有两个量级最低的元素被设置为零。

您不必设置 `pruning_schedule` 参数。默认情况下，剪枝掩码是在第一步定义的，并且在训练期间不会更新。

In [ ]:
pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

定义随机剪枝的参数，目标稀疏性为 50%。

In [ ]:
pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

定义模型架构并指定要进行剪枝的层。将根据您选择的模型层应用结构化剪枝。

在下面的示例中，我们仅对部分层进行剪枝。我们对第二个 `Conv2D` 层和第一个 `Dense` 层进行剪枝。

请注意，第一个 `Conv2D` 层不能被结构化剪枝。要进行结构化剪枝，它应该有多个输入通道。我们改用随机剪枝来对第一个 `Conv2D` 层进行剪枝。

In [ ]:
model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

训练并评估模型。

In [ ]:
batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

移除剪枝包装器，这样在将其转换为 TensorFlow Lite 格式时不会将其包含在模型中。

In [ ]:
model = tfmot.sparsity.keras.strip_pruning(model)

## 将模型转换为 TFLite 格式

In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

## 呈现并检查权重

现在，我们来呈现使用 2 x 4 稀疏性剪枝的 `Dense` 层中的权重结构。从 TFLite 文件中提取权重。

In [ ]:
# Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

为了验证我们是否选择了正确的已进行剪枝的层，打印权重张量的形状。

In [ ]:
print(f"Shape of Dense layer is {tensor_data.shape}")

现在，我们来呈现权重张量的一个小子集的结构。最后一个维度中的权重张量的结构是稀疏的，使用 `(2,4)` 模式：四个元素中有两个元素为零。为了使呈现更清晰，我们将所有非零值替换为 1。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)

定义用于绘制分隔线的辅助函数以清晰地查看结构。

In [ ]:
def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

现在，呈现权重张量的子集。

In [ ]:
plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

呈现 `Conv2D` 层的权重。在最后一个通道（类似于 `Dense` 层）中应用结构稀疏性。如上所述，只有第二个 `Conv2D` 层被结构化剪枝。

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
tensor_name = 'structural_pruning/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[1]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

与 `Dense` 层的权重类似，内核的最后一个维度具有 (2, 4) 结构。

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

我们看一下那些随机剪枝的权重看起来如何。我们提取它们并显示权重张量的子集。

In [ ]:
# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tensor_data.shape[0],tf.reduce_prod(tensor_data.shape[1:])])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 0, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Unstructed pruned weights for Conv2D layer")
plt.show()

TensorFlow Model Optimization Toolkit 包含一个 Python 脚本，可用于检查给定 TFLite 文件中模型的哪些层具有结构化剪枝权重：[`check_sparsity_m_by_n.py`](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py)。以下命令演示了如何使用此工具检查特定模型中的 2 x 4 稀疏性。

In [ ]:
! python3 ./tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py --model_tflite=pruned_model.tflite --m_by_n=2,4
